In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install transformers langchain datasets evaluate
!pip install --upgrade transformers
!pip install langchain-community
!pip install langchain-experimental
!pip install "langserve[all]"
!pip install langchain-cli
!pip install spacy
!pip install sklearn-som
!pip install MiniSom
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 19.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 41.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 7.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10:
      Successfully uninstalled orjson-3.9.10
ERROR: pip's dependency resolver does no

In [8]:
!pip install wandb
!pip install fastai

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [10]:
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
import transformers

In [11]:
from huggingface_hub import notebook_login

notebook_login()

In [12]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")
from datasets import load_dataset, load_metric, DownloadMode
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, Trainer, DataCollatorWithPadding, DataCollatorForLanguageModeling, TrainingArguments, pipeline, BitsAndBytesConfig

2024-02-29 19:10:15.196301: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-29 19:10:15.196456: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-29 19:10:15.371136: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [14]:
# Sample-based 
gen_config = GenerationConfig(do_sample=True, temperature=0.7)

In [16]:
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

In [17]:
model = AutoModelForCausalLM.from_pretrained("aboonaji/llama2finetune-v2")
model.gen_config = gen_config
model.quantization_config=quantization_config
tokenizer = AutoTokenizer.from_pretrained("aboonaji/llama2finetune-v2")

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [19]:
# Load dataset
dst = load_dataset("gamino/wiki_medical_terms",name="en-UK", split="train[6860]")

# Specify the download path
#download_path = "D:\AI Projects\data\med_wiki"
# Split dataset into train and test sets
train_dataset, test_dataset = dst["train"].train_test_split(test_size=0.2)

# Tokenization function
def tokenize(example):
    return tokenizer(example["term"], truncation=True)

# Aboulomania - mental disorder characteristed by indecisiveness.
tokenized_dataset = dst.map(tokenize, batched=True)
# Tokenize datasets
tokenized_train_dataset = train_dataset.map(tokenize, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize, batched=True)

print(tokenized_dataset)

# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
tokenized_dataset = data_collator(tokenized_dataset)

ValueError: Invalid pattern: '**' can only be an entire path component